In [22]:
import openai
import pandas as pd
import numpy as np
import string
import json

In [23]:
#text embedding
modelTE1="second-state/Nomic-embed-text-v1.5-Embedding-GGUF/nomic-embed-text-v1.5-f16.gguf"
modelTE2="nomic-ai/nomic-embed-text-v1.5-GGUF/nomic-embed-text-v1.5.Q5_K_M.gguf"
#llms
model1="dranger003/SFR-Embedding-Mistral-GGUF/ggml-sfr-embedding-mistral-q4_k_m.gguf" #json ko, ha generat variacions i 8k te texte, generativa ko (no correspon a la pregunta i triga molt i consumeix molt
model2="TheBloke/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q6_K.gguf" #json ok no tradueix be la frase, triga poc (11seg)
model3="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf" #json ok frase ok (11seg)
model4="NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/resolve/main/Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf" #instruccions afegeix explicacions, s'inventa les fonts de la generativa respon ok. tradueix a tots els idiomes
model5="lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf" # no torna contingut
model6="TheBloke/phi-2-GGUF/phi-2.Q5_K_M.gguf" # for code generation proposal, ha generat codi i json mal format sense "" o resposta sense sentit repeteix els documents
model7="bartowski/Starling-LM-7B-beta-GGUF/Starling-LM-7B-beta-IQ4_XS.gguf" #No es centra en json, en la generativa s'embucla amb texte sense sentit (commes i salts de linia) i em faltes d'ortografia no encerta les fonts, afegeix explicacions, consum alt i lent.
model8="TheBloke/stablelm-zephyr-3b-GGUF/stablelm-zephyr-3b.Q8_0.gguf" #json ok. generativa a mitges, fonts ok
model9="Aryanne/Mamba-gpt-3B-v4-ggml-and-gguf/resolve/main/q5_1-gguf-mamba-gpt-3B_v4.gguf" #no hace json, la generativa buida i el json d'idioma diu que ha de ser en anglés
model10="mradermacher/Mixtral_AI_MegaTron-GGUF/resolve/main/Mixtral_AI_MegaTron.Q4_K_M.gguf"#json idioma detectat español, traducció buida ok, json clasificación ok, genrativa ok, molt rápido. Altres proves no respon res ( ni classificació ni generativa)
model11="DevQuasar/falcon2-11B-GGUF/resolve/main/falcon2-11B.Q4_0.gguf" #json ok, generativa triga molt i contesta sense sentit caràcters extranys
model12="bartowski/WizardLM-2-7B-GGUF/resolve/main/WizardLM-2-7B-Q8_0.gguf" #segon millor LLM dp llama3? altres proves afegeix <p> al final i no acaba, s'ha hagut de parar
model13="bartowski/Yi-1.5-34B-Chat-GGUF/resolve/main/Yi-1.5-34B-Chat-IQ1_M.gguf" #molt lent i consumeix molt processament y contesta ko s'enrolla i explica altres coses, classificació no torna json amb format {}, i generativa ko. 
model14="failspy/kappa-3-phi-3-4k-instruct-abliterated-GGUF/ggml-model-f16.gguf" #bo com llama3. Altres proves json ok, s'inventa preguntes i respostes, generativa ko, triga molt i consumeix molt
model15="ecastera/eva-mistral-7b-spanish-GGUF/resolve/main/Turdus-trained-20-int4.gguf" #json ko, generativa ko, no contesta res molt ràpid
model16="zhengr/MixTAO-7Bx2-MoE-v8.1-GGUF/mixtao-7bx2-moe-v8.1.Q4_K_M.gguf" #json ok, generativa ko, no pot trobar la resposta, molt consum de processador i molt lent.



In [24]:
#user_query = "¿Cómo se pide que me estampen una Tarjeta?"
user_query  = 'que cuota anual tiene una tarjeta de debito'
#user_query =  'no puedo condonarme una cuota'

In [25]:
from sentence_transformers import SentenceTransformer

def embeddings_GIST(docs: list[str]):
    model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0")
    embeddings = model.encode(docs, convert_to_tensor=False)
    return embeddings

In [26]:
# Defining cosine similarity 
def compute_cosine_similarity(a,b): 
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return (cos_sim)

In [27]:
df_embed_uoc = pd.read_pickle('data/variaciones_all_comp_embedding_uoc_20240427.pkl')
df_embed_uoc.head()

,CODIGO_FAQ,embeddings_mini,embeddings_bert,embeddings_spacy,embeddings_mxbai,embeddings_GIST,motivo
0,433243,"[-0.049297784, 0.06306262, -0.052778855, -0.04...","[-0.049297784, 0.06306262, -0.052778855, -0.04...","[0.06396655, -1.0102829, 0.100738235, 1.127394...","[0.02481748, -0.03403738, -0.14437212, 0.72762...","[-0.030527227, 0.0026669737, 0.01400228, 0.012...",NaN
1,419915,"[-0.026050719, -0.07427596, -0.05052995, -0.07...","[-0.026050719, -0.07427596, -0.05052995, -0.07...","[0.84611166, 0.28720096, 0.834923, -1.2378342,...","[0.48073822, -0.018162876, 0.223723, 0.5368666...","[-0.043238733, -0.026242362, 0.0061765215, -0....",NaN
2,421998,"[-0.12729552, 0.06905968, -0.080994666, -0.075...","[-0.12729552, 0.06905968, -0.080994666, -0.075...","[0.35114053, -0.24000733, 0.6779311, -0.876988...","[-0.028965719, 0.4119485, 0.04523738, 0.877965...","[-0.0527733, 0.0033994697, 0.02993245, -0.0139...",NaN
3,423029,"[-0.1008265, 0.026075443, -0.042023044, -0.106...","[-0.1008265, 0.026075443, -0.042023044, -0.106...","[-0.12797742, -0.97483176, 0.7096605, 0.197163...","[0.49374753, -0.26194438, 0.089569956, 0.30814...","[-0.031788237, 0.008151135, 0.04830003, 0.0147...",NaN
4,422618,"[-0.06829783, 0.0648448, -0.07408309, -0.08305...","[-0.06829783, 0.0648448, -0.07408309, -0.08305...","[0.054455537, -0.6434433, 0.06104442, 0.659452...","[0.031954594, -0.016352661, -0.09663524, 0.226...","[-0.03212525, -0.019531578, 0.03505223, 0.0109...",NaN


In [28]:
e = embeddings_GIST(user_query)
print (e)
df_embed_uoc['score_GIST'] = df_embed_uoc['embeddings_GIST'].apply(lambda x: compute_cosine_similarity(x, e))

[-4.71228100e-02 -5.33197948e-04  2.88016889e-02 -1.66551918e-02
  5.36670862e-03 -1.07323406e-02  7.11195469e-02  6.23402745e-02
  2.08508987e-02  7.14377081e-03 -3.97571065e-02 -2.81380024e-02
  1.75720789e-02  1.72490831e-02  3.08912974e-02 -4.33316194e-02
 -4.14482877e-02  4.29461431e-03 -1.14790155e-02  7.21155480e-03
  8.71606916e-02 -2.66336054e-02 -7.37904310e-02 -1.52886854e-02
 -1.08038378e-03  5.24382107e-02 -2.39456259e-02 -2.39863526e-02
 -1.43018384e-02 -1.63210317e-01  7.43647944e-03  9.69498511e-03
  8.41220096e-03 -8.06301832e-02  6.02720343e-02 -7.06994236e-02
 -4.90086116e-02  3.28563191e-02  3.28282192e-02  3.26035768e-02
  8.19714833e-03  2.25679222e-02 -3.67723852e-02 -8.35621357e-02
  1.73812341e-02 -2.26683319e-02  1.06337704e-02  7.23597780e-02
  6.78244559e-03  1.54558979e-02  4.44126315e-03  2.61907210e-03
 -3.62756625e-02  2.09712591e-02 -3.14896107e-02  3.17197666e-02
  2.77330372e-02  2.56121214e-02  5.59372529e-02  2.31202040e-02
  5.48799848e-03 -4.45463

In [29]:
# Get index of the 5 highest score_GIST's rows 
indices_max_scores_GIST= df_embed_uoc['score_GIST'].nlargest(2).index

# print scores and codes
codes_max_scores_GIST = df_embed_uoc.loc[indices_max_scores_GIST, ['CODIGO_FAQ','score_GIST']] 
print(codes_max_scores_GIST)

     CODIGO_FAQ  score_GIST
104      418183    0.894896
122      418606    0.885208


In [30]:
df_corpus_uoc = pd.read_pickle('data/corpus_con_variaciones_20240427_uoc_ofuscado.pkl')
df_corpus_uoc.head()
df_corpus_docs = df_corpus_uoc[['CODIGO_FAQ','title_found','body texto']]
df_corpus_docs.head()
#documents = [df_corpus_uoc["CODIGO_FAQ"=="422766", ]
#{"993366716":"Se puede enviar una tarjeta al extranjero --- No existe un canal para envío automático de tarjetas al extranjero."}


,CODIGO_FAQ,title_found,body texto
0,433243,Visa Lidl para empleados del grupo Lidl,¿A quién está dirigida? La Visa Lidl Empleado ...
1,419915,¿Cómo puedo anular el cobro de una liquidación...,¿Cómo puedo anular el cobro de una liquidación...
2,421998,¿Cómo puedo modificar un contrato de tarjeta p...,¿Cómo puedo modificar un contrato de tarjeta p...
3,423029,¿Cómo modificar la forma o importe de cobro de...,¿Cuál es la comisión por recargar una tarjeta ...
4,422618,¿Cuáles son los plazos de recepción de las tar...,¿De que depende el plazo de recepción de las t...


In [31]:
def get_documents_by_code(codes, df_docs):
    return df_corpus_docs[df_docs['CODIGO_FAQ'].isin(codes)]    

In [32]:
def get_top_documents(codes, df_docs):
    added_docs = 0
    docs = ''
    document = get_documents_by_code(codes, df_docs)
    print (document)
    
    for idx, doc in document.iterrows():
        docs += "#Documento " + str(doc['CODIGO_FAQ']) + ": "  + doc['title_found'] + " - " + doc['body texto'] + "\n"
        added_docs += 1
    return docs

In [33]:
prompt_docs = get_top_documents(codes_max_scores_GIST['CODIGO_FAQ'].tolist(),df_corpus_docs)
print(prompt_docs)

     CODIGO_FAQ                                        title_found  \
106      418183    Modificaciones en el alta de tarjetas de débito   
124      418606  Límite de crédito de 200 € para reintegros en ...   

                                            body texto  
106  Se cierra la contratación de VISA DÉBITO para ...  
124  Este importe se ofrecerá al cliente cuando al ...  
#Documento 418183: Modificaciones en el alta de tarjetas de débito - Se cierra la contratación de VISA DÉBITO para clientes entre 30 y 60 años. La operativa recomendará la contratación de VISA Express con límite recomendado : clientes con límite preconcedido, análisis de riesgo o suficiente límite de crédito. Express con 200 € de límite de crédito : clientes sin suficiente límite disponible para tarjeta de crédito, con probabilidad de mora inferior a 10 y sin impagados en UOCBank ni externos. Express con 500 € (sin necesidad de elevar al CARP): clientes sin impagados en UOCBank. Express con un límite de crédito

In [189]:
prompt2="Eres un asistente que responde consultas a los empleados del banco UOCBank sobre documentación interna. Las consultas que te llegan son dudas que te ha hecho un empleado porque un cliente ha ido a la oficina y tiene alguna duda y el empleado no la sabe responder.\
        Consulta del empleado: {user_query}\
        Tu objetivo es responder la anterior pregunta de forma clara y breve siguiendo las siguientes instrucciones:\
            - El formato de salida tiene que ser SIEMPRE solo un json con dos campos, el campo \"respuesta\" y el campo \"fuentes\", sin nada más de texto añadido.\
            - El campo respuesta debe contener la información necesaria para responder la pregunta y el campo fuentes una lista de los docuementos de donde se ha obtenido la respuesta.\
            - Toda la información de la respuesta generada tiene que provenir UNICAMENTE de los documentos facilitados marcados entre triples #.\
            - Usa el significado y el contexto entero de la consulta del empleado, si no estas completamente seguro de que la respuesta está en los documentos debes indicarlo en el campo respuesta y dejar las fuentes vacías.\
            - Si consideras que no tienes suficiente información específica en los documentos, di que no has podido contestar en el campo respuesta y deja el campo fuentes como una lista vacía: \"[]\".\
            - No asumas nada que no esté explícito en la consulta o en los documentos, es crítico que no te inventes nada y solo respondas con la información en los documentos.\
            - Nunca inventes respuestas ni información que no utilicen los documentos. Debes usar la información de cada documento en su contexto, sin sacar de contexto ninguna frase.\
            - En el campo respuesta no debes añadir el origen de la información, es decir nunca añadas referencias del tipo: \"Más información en el documento XXXXX\" o \"Puedes consultar el documento XXXXXX\"\
            - El campo fuentes debe consistir de los identificadores de los documentos utilizados para la respuesta en formato lista, p. ej. [\"817231\", \"444444\"]. Solo tienes que añadir a la lista los documentos que hayas usado su contenido para la respuesta. Si no usas el contenido de un documento para la respuesta no debes añadirlo a la lista.\
            - En caso de haber encontrado fuentes, debes ordenarlas de mas a menos importantes en la respuesta.\n- La respuesta puede tener como máximo 100 palabras. Si la respuesta es muy larga, resúmela para facilitar la información.\
            - El campo respuesta debe llevar entre las palabras clave los marcadores de HTML <b> y <\\b> para que aparezcan en negrita más adelante. También puede llevar marcadores tipo <BR>, <li>, etc para que sea una salida mas visible.\
        Los documentos para generar la respuesta son los siguientes:\
            ###Documentos:\
            {documents}\
            ###\
        El formato de salida tiene que ser SIEMPRE el siguiente:\
            {\
                    \"respuesta\": \"Respuesta\",\
                    \"fuentes\": [\"123456\", \"658194\", ...]\
            }"

In [13]:
prompt1="Eres un asistente que responde consultas a los empleados del banco UOCBank sobre documentación interna. \
        Consulta del empleado: {user_query}\
        Tu objetivo es responder la anterior pregunta de forma clara y breve siguiendo las siguientes instrucciones:\
            - El formato de salida tiene que ser SIEMPRE solo un json con dos campos, el campo \"respuesta\" y el campo \"fuentes\", sin nada más de texto añadido.\
            - El campo respuesta debe contener la respueta a la pregunta y el campo fuentes una lista de los docuementos de donde se ha obtenido la respuesta.\
            - Toda la información de la respuesta generada tiene que provenir UNICAMENTE de los documentos facilitados marcados entre triples #.\
            - Si no estas completamente seguro de que la respuesta está en los documentos debes indicarlo en el campo respuesta y dejar el campo fuentes como una lista vacía: \"[]\".\
            - No asumas nada que no esté explícito en la consulta o en los documentos, es crítico que no te inventes nada y solo respondas con la información en los documentos.\
            - Nunca inventes respuestas ni información que no utilicen los documentos.\
            - El campo fuentes son los identificadores de los documentos utilizados para la respuesta en formato lista, p. ej. [\"817231\", \"444444\"].\
            - La respuesta puede tener como máximo 100 palabras. Si la respuesta es muy larga, resúmela para facilitar la información.\
            - La respuesta debe ser en el idioma en el que está la consulta.\
        Los documentos para generar la respuesta son los siguientes:\
            ###Documentos:\
            {documents}\
            ###\
        El formato de salida tiene que ser SIEMPRE el siguiente:\
            {\
                    \"respuesta\": \"Respuesta\",\
                    \"fuentes\": [\"123456\", \"658194\", ...]\
            }"

In [40]:
prompt_simple = "You are an assistant that responds to queries from UOCBank employees about internal documentation. \
The queries you receive are doubts that an employee has because a customer has come to the office and has a question that the employee cannot answer.\
Employee query: {user_query}\
Your goal is to answer the previous question clearly and concisely following the following instructions:\
- The output format must ALWAYS be a single json with two fields, the \"response\" field and the \"sources\" field, without any additional text.\
- The \"response\" field must contain the necessary information to answer the question and the \"sources\" field a list of the documents from which the answer has been obtained.\
- All the information in the generated response must come ONLY from the provided documents marked between triple #.\
- Use the meaning and the entire context of the employee's query. If you are not completely sure that the answer is in the documents, you must indicate it in the \"response\" field and leave the sources field empty.\
- If you consider that you do not have enough specific information in the documents, say that you have not been able to answer in the \"response\" field and leave the sources field as an empty list: \"[]\".\
- Do not assume anything that is not explicit in the query or in the documents. It is critical that you do not make anything up and only respond with the information in the documents.\
- Never invent answers or information that is not used in the documents. You must use the information from each document in its context, without taking any phrase out of context.\
- Do not add the origin of the information in the \"response\" field, that is, never add references like \"More information in document XXXXX\" or \"You can consult document XXXXXX\".\
- The sources field must consist of the identifiers of the documents used for the answer in list format, e.g. [\"817231\", \"444444\"]. You only have to add to the list the documents whose content you have used for the answer. If you do not use the content of a document for the answer, you must not add it to the list.\
- If you have found sources, you must order them from most to least important in the response.\n- The response can have a maximum of 100 words. If the response is too long, summarize it to provide the information.\
- The \"response\" field must include HTML markers <b> and <\\b> to make the text bold later. It can also include markers like <BR>, <li>, etc. to make the output more visible.\
The documents to generate the response are the following:\
    ###Documents:\
    {documents}###\
The output format must ALWAYS be as follows:\
    {\
        \"response\": \"Answer\",\
        \"sources\": [\"123456\", \"658194\", ...]\
    }.\
The answer MUST BE ALWAYS in SPANISH."


In [41]:
final_prompt = prompt_simple.replace("{user_query}", user_query)
prompt_data = final_prompt.replace("{documents}", prompt_docs)

In [41]:
user_query =  "¿Cóm es demana una targeta?"

In [57]:
user_query = "Quin es el cost de la Visa Classic?"

In [68]:
user_query="Which is the cost of Visa Classic?"

In [129]:
user_query = "hola"

In [46]:
prompt3="""Un usuario ha introducido la siguiente frase:\"""" + user_query + """\"
    Devuelve SOLO un json plano con 2 únicos campos:
    'idioma': el idioma detectado. Las opciones son Catalán, Español, Inglés
    'traduccion_castellano': traduce aquí la frase del usuario, si el idioma no es Español, si es Español deja el campo vacío "".
    Si tienes que traducir, deja sin modificar las palabras que identifiques como nombres propios o palabras clave, por ejemplo eBillete, Pay, Apple, Express, etc.
    Un ejemplo de respuesta sería:{'idioma': 'Catalán', 'traduccion_castellano': '¿Cómo se pide una tarjeta?'}. Contesta sólo con el JSON y NO añadas explicaciones."""

In [69]:
prompt3="""Translate \""""+ user_query + """\" into spanish 
   Return ONLY a plain json with 2 unique fields:
   -'language': the detected language. The options are Catalan, Spanish, English.
   -'translation_spanish': translate the user's sentence here, if the language is not Spanish, if it is Spanish leave the field empty "".
   If you have to translate, leave unchanged the words that you identify as proper names or keywords, for example eBillete, Pay, Apple, Express, etc. 
   An example of response would be: {'language': 'Catalan', 'translation_spanish': 'How to request a card?'}. Reply only with the JSON and DO NOT add explanations."""

In [54]:
prompt3="A user has entered the following sentence: '"""+ user_query + """'.
Return ONLY a plain json with 2 unique fields:
'language': the detected language. The options are Catalan, Spanish, English.
'translation_spanish': translate the user's sentence here, if the language is not Spanish, if it is Spanish leave the field empty \"\". 
If you have to translate, leave unchanged the words that you identify as proper names or keywords, for example eBillete, Pay, Apple, Express, etc. 
An example of response would be: {'language': 'Catalan', 'translation_spanish': 'How to request a card?'}. Reply only with the JSON and DO NOT add explanations."""

In [70]:
prompt_data = prompt3

In [14]:
prompt = prompt1

In [15]:
final_prompt = prompt.replace("{user_query}", user_query)
prompt_data = final_prompt

In [16]:
prompt_data = final_prompt.replace("{documents}", prompt_docs)

In [24]:
prompt4="Eres un asistente para UOCBank, el cliente ha hecho una pregunta y no se ha encontrado documentación para poder responder. ¿Puedes contestar al usuario disculpandote y pedir que reformule la pregunta?"
prompt_data = prompt4

In [26]:
user_query = "¿Cuánto cuesta la vanguardia?"
prompt5 = "Eres un asistente para UOCBank, te han hecho una pregunta y se ha detectado que tema es diferente de \"Tarjetas\" y sólo sabes responder sobre este tema. ¿Puedes contestar al usuario disculpándote y pedir que haga otra pregunta sobre el tema \"Tarjetas\"?"
prompt = prompt5
prompt_data = prompt.replace("{user_query}", user_query)

In [22]:
#user_query = 'que coste tiene una tarjeta de debito'
#user_query = '¿Cuánto cuesta la vanguardia?'
#user_query = '¿Qué precio tiene el periódico?'
#user_query = '¿Cómo se pide que me estampen una Tarjeta?'
#user_query = '¿Cómo se pide una visa Oro?'
#user_query = '¿Cuál es mejor banco? El Santander o el BBVA?'
#user_query  = '¿Quién va a ganar las elecciones calatalanas?'
#user_query = 'que día más bonito'
user_query = 'no puedo condonarme una cuota'
#prompt6 = "Eres un asistente del banco UOCBank y solo debes responder a preguntas de la temática  \"Tarjetas\", que incluye dudas y consultas sobre operaciones relacionadas con tarjetas, motivos de denegación, compras, pagos realizados con tarjetas, liquidaciones, pagos mensuales, fraccionamientos, pagos a plazos, renovaciones, bajas, límites de crédito, límites operativos, consultas sobre pin, pagos móvil, Tarjeta eBillete, ahorra el cambio, Paypal, UOCBank Sleep, compras pagos por internet, contactless, apple pay, samsung pay, google pay, visa, master card. Cuando veas consultas del tipo \"Motivo XXX\" también deben ser incluidos en esta temática. Cualquier otra temática relacionada con el banco pero no de tarjetas deben ser etiquetadas como \"OTHER\". Algunas de las temáticas que no deben ser incluidas en Tarjetas son: temas no relacionados con el banco (chit-chat), cajeros, transferencias, etc.\nEl formato de la salida SIEMPRE tiene que ser SOLO un JSON con 3 campos:\n-\"detect_change\": 0 en caso de que se está en la temática de tarjetas y 1 si no. Las UNICAS opciones son: 0 o 1\n-\"topic\": si se trata de tarjetas se debe indicar \"TARJETA\", y \"OTHER\" en otro caso. Las UNICAS opciones son:  \"TARJETA\" o \"OTHER\"\n-\"user_query\": Copia exacta de la pregunta del empleado.\nEjemplos de formato de salida:\nEjemplo 1: model output: {\"detect_change\": 0, \"topic\": \"TARJETA\", \"user_query\": \"cuáles son los costos asociados al pago con la tarjeta prepago\"}\nEjemplo 2: model output: {\"detect_change\": 1, \"topic\": \"OTRO\", \"user_query\": \"alta de fondo de inversion\"}\nLa pregunta del empleado es la siguiente: {user_query}"
#prompt6 = "Eres un asistente del banco UOCBank y solo debes responder a preguntas de la temática  \"Tarjetas\". Cualquier otra temática relacionada con el banco pero no de tarjetas deben ser etiquetadas como \"OTHER\".\nEl formato de la salida SIEMPRE tiene que ser SOLO un JSON con 3 campos:\n-\"detect_change\": 0 en caso de que se está en la temática de tarjetas y 1 si no. Las UNICAS opciones son: 0 o 1\n-\"topic\": si se trata de tarjetas se debe indicar \"TARJETA\", y \"OTHER\" en otro caso. Las UNICAS opciones son:  \"TARJETA\" o \"OTHER\"\n-\"user_query\": Copia exacta de la pregunta del empleado.\nEjemplos de formato de salida:\nEjemplo 1: model output: {\"detect_change\": 0, \"topic\": \"TARJETA\", \"user_query\": \"cuáles son los costos asociados al pago con la tarjeta prepago\"}\nEjemplo 2: model output: {\"detect_change\": 1, \"topic\": \"OTRO\", \"user_query\": \"alta de fondo de inversion\"}\nLa pregunta del empleado es la siguiente: {user_query}"

prompt6 = """Categoriza: '""" + user_query + """' en 'TARJETA' u 'OTRO'
    Devuelve un JSON con SOLO dos campos:
    -'topic': la temática de la frase del usuario, si es sobre tarjetas la respuesta tiene que ser TARJETA, de cualquier otra tematica la respuesta tiene que ser OTHER. Las opciones son TARJETA, OTHER.
    -'user_query': la frase del usuario. 
    Ejemplos de respuesta:
        Ejemplo 1: {'topic': 'TARJETA', 'user_query': 'cuáles son los costos asociados al pago con la tarjeta prepago'}. 
        Ejemplo 2: {'topic': 'OTHER', 'user_query': 'alta de fondo de inversion'}.
    Contesta SOLO con el JSON y NO añadas ninguna explicación más."""
prompt6_en = """Categorize the topic of: '""" + user_query + """' in one of these options: 'CARD' or 'OTHER' 
    Return a JSON with ONLY two fields:
    -'topic': the topic of the user's sentence, if it is about cards the answer has to be CARD, of any other topic the answer has to be OTHER. The options are CARD, OTHER.
    -'user_query': the user's sentence. 
    Response examples:
        Example 1: {'topic': 'CARD', 'user_query': 'what are the costs associated with payment with the prepaid card'}. 
        Example 2: {'topic': 'OTHER', 'user_query': 'investment fund registration'}.
    Answer ONLY with the JSON and DO NOT add any further explanation."""
prompt_6_en = """A user has entered the following sentence: '"""+ user_query + """'. Return a JSON with ONLY two fields:\n-'topic': the topic of the user's sentence, if it is about cards the response has to be CARD, for any other topic the response has to be OTHER. The options are CARD, OTHER.\n-'user_query': the user's sentence.\nExamples of response:\nExample 1: {\"topic\": \"CARD\", \"user_query\": \"what are the costs associated with payment with the prepaid card\"}\nExample 2: {\"topic\": \"OTHER\", \"user_query\": \"investment fund registration\"}\nReply ONLY with the JSON and DO NOT add additional explanations."""
prompt_data = prompt_6_en
#prompt_data = prompt.replace("{user_query}", user_query)

In [20]:
conversation = {
  "id": "665355904c65fcc0e860dcba",
  "user": "662eac875a187ab12cafc923",
  "conversation": [
    {
      "query": "que cuota anual tiene una tarjeta de debito",
      "response": "system<|end_header_id|>\n{:output}\n{\n    \"response\": \"<b>Cuota anual</b> de una tarjeta de débito es gratuita hasta los 26 años. A partir de los 26 años se realizarán acciones centralizadas para sustituir la tarjeta por una tarjeta de crédito o una Express Visa débito Oro: <b>50€</b>. La tarjeta es gratuita el primer año. Para los siguientes años será gratuita, si el cliente domicilia la pensión y realiza compras por más de 300 € por tarjeta.\",\n    \"sources\": [\"418183\"]\n}"
    },
    {
      "query": "¿Cómo puedo pagar la declaración de la Renta con una Tarjeta?",
      "response": "system<|end_header_id|>\nThanks for your patience. Unfortunately, I'm a large language model, my responses are limited to the information provided by my training data, and I don't have enough specific information in the documents you've given me to answer your query.\n\nHowever, I"
    },
    {
      "query": "¿Cóm puc pagar la declaració de la renda amb una targeta?",
      "response": "assistant<|end_header_id|>\n\nLo siento mucho, pero después de revisar nuestra base de conocimientos y documentos, no hemos encontrado información que responda exactamente a su pregunta. Me gustaría pedirle su ayuda para clarificarla un poco más. ¿Podría reformular la pregunta o proporcionarme más detalles sobre lo que está buscando? Esto me ayudará a brindarle una respuesta más precisa y útil. Por favor, disculpe el incómodo y agradezco su colaboración."
    }
  ]
}
prompt_data = """Summarize the following conversation""" + json.dumps(conversation) +""". Respond with the summary IN SPANISH"""

In [59]:
prompt_hallu ="Return only the sentences that are hallucinations in the response. If there are no hallucinations, return an empty string:\"\". The user query is: {user_query}. Original text:{documents} and the response is: {response}."


In [60]:
documents = """A través del perfil de fraccionamiento se gestionan las condiciones que han de cumplir las operaciones para poder fraccionarse de forma automática. 
Alta del perfil
expandir
Al dar de alta el perfil de fraccionamiento automático se debe indicar el importe a partir del cual se fraccionarán las operaciones y qué modalidad de fraccionamiento se desea. A partir de ese momento, todas las operaciones que cumplan con los requisitos informados, se fraccionarán automáticamente. 
En la modalidad de fraccionamiento con cobro de comisión, el tramo va de 210€ a 500€ por lo tanto, no se fraccionará automáticamente, las operaciones de importe superior a 500€.
Se debe tener en cuenta que las operaciones realizadas con anterioridad al alta del perfil no se fraccionarán y si el cliente las quiere también fraccionadas tendrá que hacerlo manualmente. 
Un contrato con forma de pago aplazado no permite perfil de fraccionamiento. 
¿Dónde se encuentra esta operativa? 
Productos > Tarjetas y comercios > Tarjetas >Condiciones > Forma de pago > Fraccionamiento automático de operaciones. 
1
Selecciona la tarjeta que se desea dar de alta el perfil de fraccionamiento automático.
2
Indicar M para modificar
3
Informar los siguientes datos:
  Indicar el importe a partir del cual se fraccionará.   Indicar la modalidad de fraccionamiento (pulsando F8 o Av. Pág accedemos a más modalidades de fraccionamiento).    Indicar S en esta opción si el cliente tiene operaciones fraccionadas y desea que se le apliquen los cambios a la modalidad del perfil. 
Modificación de condiciones
expandir
Se pueden modificar las condiciones del perfil de fraccionamiento: el importe a partir del cual se quiere fraccionar y la modalidad de fraccionamiento. 
Se debe tener en cuenta que las operaciones realizadas con anterioridad a la modificación mantendrán las condiciones antiguas. Si se ha modificado la modalidad de fraccionamiento y se quiere que las operaciones fraccionadas con anterioridad recojan los cambios se tendrá que indicar que sí en Repercutir cambio modalidad en operaciones que están actualmente fraccionadas. 
No se podrá repercutir sobre una operación si eso implica una refinanciación de las condiciones anteriores de la operación.
¿Dónde se encuentra esta operativa? 
Productos > Tarjetas y comercios > Tarjetas > Condiciones > Forma de pago > Fraccionamiento automático de operaciones. 
1
Seleccionar la tarjeta sobre la que se desea dar de alta modificar el perfil de fraccionamiento.
2
 Indicar M para modificar los datos deseados. 
3
Informar los siguientes datos: Indicar el importe a partir del cual se fraccionará. Indicar la modalidad de fraccionamiento (pulsando F8 o Av. Pág accedemos a más modalidades de fraccionamiento).  Indicar S en esta opción si el cliente tiene operaciones fraccionadas y desea que se le apliquen los cambios a la modalidad del perfil. 
Baja del perfil
expandir
Dar de baja el perfil de fraccionamiento implica que a partir de ese momento no se fraccionarán las operaciones, pero las operaciones fraccionadas con anterioridad a la baja mantienen las condiciones del perfil. 
¿Dónde se encuentra esta operativa? 
Productos > Tarjetas y comercios > Tarjetas > Condiciones > Forma de pago > Fraccionamiento automático de operaciones. 
1
Seleccionar la tarjeta sobre la que se desea dar de baja el perfil de fraccionamiento.
2
Seleccionar con B la tarjeta en la que daremos de baja el perfil de fraccionamiento.
3
Si se desea desfraccionar las operaciones fraccionadas, marcar S; en caso contrario, marcar N."""
#user_query = "\"mi tarjeta es gratis pero la de mi mujer no\""
user_query = "\"¿Cómo activo el fraccionamiento automático?\""
#response="\"La tarjeta de débito que se le puede entregar a tu mujer no es gratuita. Según el documento 446516, la edad mínima para tener una tarjeta de débito es de 12 años, y en caso de producto de empresa, el mínimo son 16 años. En las cuentas de menores, el titular tiene que ser únicamente el menor, y obligatoriamente tiene que haber, al menos, un representante legal informado en la cuenta.\""
response="\"Para dar de alta el perfil de fraccionamiento automático, debes seleccionar la tarjeta que deseas y indicar el importe a partir del cual se fraccionará y qué modalidad de fraccionamiento se desea. A partir de ese momento, todas las operaciones que cumplan con los requisitos informados, se fraccionarán automáticamente.\""

In [61]:
prompt_hallu = prompt_hallu.replace("{user_query}", user_query)
prompt_hallu = prompt_hallu.replace("{documents}", documents)
prompt_hallu = prompt_hallu.replace("{response}", response)
prompt_data = prompt_hallu

In [62]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

print ("Prompt: ", prompt_data)
print("*****CALLING MODEL *****")
completion = client.chat.completions.create(
  model=model3,
  messages = [{ "role": "system", "content": prompt_data }],
  temperature=0.0,
  #max_tokens=100,
  #stream=True,
)
'''
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
'''
print("Result:",completion)
print(completion.choices[0].message.content)


Prompt:  Return only the sentences that are hallucinations in the response. If there are no hallucinations, return an empty string:"". The user query is: "¿Cómo activo el fraccionamiento automático?". Original text:A través del perfil de fraccionamiento se gestionan las condiciones que han de cumplir las operaciones para poder fraccionarse de forma automática. 
Alta del perfil
expandir
Al dar de alta el perfil de fraccionamiento automático se debe indicar el importe a partir del cual se fraccionarán las operaciones y qué modalidad de fraccionamiento se desea. A partir de ese momento, todas las operaciones que cumplan con los requisitos informados, se fraccionarán automáticamente. 
En la modalidad de fraccionamiento con cobro de comisión, el tramo va de 210€ a 500€ por lo tanto, no se fraccionará automáticamente, las operaciones de importe superior a 500€.
Se debe tener en cuenta que las operaciones realizadas con anterioridad al alta del perfil no se fraccionarán y si el cliente las qu